In [1]:
import utils
from typing import NamedTuple, Dict

## Day 7: Handy Haversacks 

[#](https://adventofcode.com/2020/day/7). We a list of rules laying out what color bags and their number can be in a particular colored bag.

We want to know how many colors can a `shiny gold` bag be in. Now this can be a bag within a bag within a bag... so it looks like a recursion or some such problem?

In [2]:
test7 = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags.""".splitlines()

inp7 = utils.get_input(7, splitlines=True)

First up, a simple function to parse an individual bag:

In [3]:
def parse_bag(txt):
    """parses an individual bag"""
    if txt == "no other bags":
        return "EMPTY", 0
    num, col1, col2, _ = txt.split(" ")
    return f"{col1} {col2}", int(num)

parse_bag('2 muted yellow bags'), parse_bag("no other bags")

(('muted yellow', 2), ('EMPTY', 0))

In [4]:
def parse_7(inp=test7) -> Dict[str, int]:
    """returns a dict of colors: dict of color and how many bags it can contain.
    add check to see if a bag is repeated if needed"""

    bags = {}

    for line in inp:
        color, rr = line.split(" bags contain ")
        bag_rules = rr.strip(".").split(", ")
        
        rules_dict = {}
        for bag_rule in bag_rules:
            col, num = parse_bag(bag_rule)
            rules_dict[col] = num

        bags[color] = rules_dict
        
    return bags

rules = parse_7()
rules

{'light red': {'bright white': 1, 'muted yellow': 2},
 'dark orange': {'bright white': 3, 'muted yellow': 4},
 'bright white': {'shiny gold': 1},
 'muted yellow': {'shiny gold': 2, 'faded blue': 9},
 'shiny gold': {'dark olive': 1, 'vibrant plum': 2},
 'dark olive': {'faded blue': 3, 'dotted black': 4},
 'vibrant plum': {'faded blue': 5, 'dotted black': 6},
 'faded blue': {'EMPTY': 0},
 'dotted black': {'EMPTY': 0}}

[Grokking Algos](https://www.manning.com/books/grokking-algorithms) which I read ages ago has a nice chapter on recursion, and the most important thing is that you need a base case which returns something, else it calls a itself with a simpler version of the problem (hopefully everything is trending to the base case so stuff gets returned finally.)

In [5]:
def contains_bag(bag, target="shiny gold", rules=rules):
    """returns True if given bag can contain the given bag"""
    
    # the base case - bag contains target directly so no need to investigate further
    if target in rules[bag]:
        return True
    
    # else see if the bags inside the given bag can contain the target
    return any([contains_bag(bag, target, rules) for bag in rules[bag] if bag != "EMPTY"])
    

contains_bag("bright white", "shiny gold", parse_7(test7))
contains_bag("dark orange", "shiny gold", parse_7(test7))


True

In [6]:
def solve_7(inp=test7, target="shiny gold"):
    rules = parse_7(inp)
    return sum(contains_bag(bag, target, rules) for bag in rules)

assert solve_7() == 4
solve_7(inp7)

246

## Part 2

Now we need to count the total number of bags inside a given bag as per the rules. This is a modification of the above to count bags contained instead of looking for a particular bag.

In [7]:
def contains_num_bags(bag="shiny gold", rules=rules):
    """returns the number of bags a bag holds, calculated recursively"""
    
    # count the bags recursively, with empty bags excluded 
    # n is the num of that bag it contains, added to n times the num of bags within that
    return sum([n + n*contains_num_bags(bag, rules) for bag, n in rules[bag].items() if bag != "EMPTY"])
    

assert contains_num_bags("shiny gold", rules=parse_7(test7)) == 32


In [8]:
def solve_7b(inp=test7, bag="shiny gold"):
    rules = parse_7(inp)
    return contains_num_bags(bag, rules)

solve_7b(inp7)

2976

**Notes**

- Recursion always does my head in, do more of it.
- figure out how to write better text parsers